In [1]:
import pandas as pd
import numpy as np

#Local_Authority_Highway_codes
lahc = ['E09000013','E09000014','E09000015']


#Load the Dataset
df = pd.read_csv('AccidentLondonBoroughs.csv')

In [2]:
#Formating tyes
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Time'] = pd.to_datetime(df['Time'], format='%H:%M')

#transform Date and Time values into ints

dateMaxMin = [df['Date'].max(), df['Date'].min()]
timeMaxMin = [df['Time'].max(), df['Time'].min()]

dateDelta = dateMaxMin[0] - dateMaxMin[1]
numOfDays = dateDelta.days

timeDelta = timeMaxMin[0] - timeMaxMin[1]
numOfMins = timeDelta.total_seconds() / 60

dateVals = []
for i in df['Date']:
    delt = i - dateMaxMin[1]
    val = delt.days
    dateVals.append(val)
    
timeVals = []
for i in df['Time']:
    delt = i - timeMaxMin[1]
    val = int(delt.total_seconds() / 60)
    timeVals.append(val)
    
df.drop(columns = ['Date', 'Time'], inplace = True)
    
df['DateVals'] = dateVals
df['TimeVals'] = timeVals


print('dateStart: ' + str(dateMaxMin[1].date()) + '\n' 
      + 'dateEnd: ' + str(dateMaxMin[0].date()) + '\n' 
      + 'totalDays: ' + str(numOfDays) + '\n' 
      + 'timeStart: ' + str(timeMaxMin[1].time()) + '\n'
      + 'timeEnd: ' + str(timeMaxMin[0].time()) + '\n'
      + 'totalMins: ' +  str(numOfMins))

dateStart: 2005-01-01
dateEnd: 2010-12-31
totalDays: 2190
timeStart: 00:01:00
timeEnd: 23:59:00
totalMins: 1438.0


In [3]:
#get columns associated with the LAHC
data = pd.DataFrame(columns = df.columns.to_list())
dfDict = dict(df.dtypes)

for key, val in dfDict.items():
    if val == 'int64' or val == 'float64':
        data[key] = pd.to_numeric(data[key])
    
for lac in lahc :
    lacRows = df.loc[df['Local_Authority_(Highway)'] == lac]
    data = pd.concat([data, lacRows])


In [4]:
# raw, unclean, with outliers data
data.shape

(10681, 32)

In [5]:
class ColumnAttri:
    def __init__(self, name, ctype):
        self.name = name
        self.ctype = str(ctype)
        self.range = []
        self.values = []
        self.empty = 0
        self.notEmpty = 0
        
    def processCol(self, data):
        for row in data[self.name]:
            if row == -1 or str(row) == '-1':
                self.empty += 1
                continue
            self.notEmpty += 1
            
            if self.ctype == 'int64' or self.ctype == 'float64':
                if len(self.range) >= 1:
                    if self.range[0] > row:
                        self.range[0] = row
                    if self.range[1] < row:
                        self.range[1] = row
                else:
                    self.range.append(row)
                    self.range.append(row)
                    
            if self.name == 'Date' or self.name == 'Time':
                if len(self.range) >= 1:
                    if self.range[0] > row:
                        self.range[0] = row
                    if self.range[1] < row:
                        self.range[1] = row
                else:
                    self.range.append(row)
                    self.range.append(row)
                    
            if not row in self.values:
                self.values.append(row)
                
    def presentColAttri(self):
        numerics = ['float64', 'int64' , 'datetime64[ns]']
        
        if not self.ctype in numerics:
            self.range = ['Nan', 'Nan']
        attri = {'Name': self.name, 'Type': self.ctype, 'Empty': self.empty, 'Full': self.notEmpty, 'Ratio': (self.empty / 10681) * 100, 'Min': self.range[0], 'Max': self.range[1], 'Unique': len(self.values)}
        return attri
    
    def getValues(self):
        return self.values

In [6]:
def analyzeDF(dat):
    ColAttributes = []
    dataDict = dict(dat.dtypes)

    for key, val in dataDict.items():
        currCol = ColumnAttri(key, val)
        currCol.processCol(dat)
        ColAttributes.append(currCol)

    colDF = pd.DataFrame(columns=['Name','Type', 'Empty', 'Full', 'Ratio', 'Min', 'Max', 'Unique'])

    for col in ColAttributes:
        colDF = colDF.append(col.presentColAttri(), ignore_index=True)

    print(colDF.shape)
    return colDF


In [7]:
#allSampsDF = analyzeDF(df)
#allSampsDF

In [8]:
anaDF1 = analyzeDF(data)
anaDF1

(32, 8)


,Name,Type,Empty,Full,Ratio,Min,Max,Unique
0,Accident_Index,object,0,10681,0.000000,Nan,Nan,10681
1,Location_Easting_OSGR,int64,0,10681,0.000000,510800,535190,1876
2,Location_Northing_OSGR,int64,0,10681,0.000000,175590,193940,1295
3,Longitude,float64,0,10681,0.000000,-0.401384,-0.049164,7192
4,Latitude,float64,0,10681,0.000000,51.46535,51.632397,6989
5,Police_Force,int64,0,10681,0.000000,1,1,1
6,Accident_Severity,int64,0,10681,0.000000,1,3,3
7,Number_of_Vehicles,int64,0,10681,0.000000,1,8,7
8,Number_of_Casualties,int64,0,10681,0.000000,1,9,7
9,Day_of_Week,int64,0,10681,0.000000,1,7,7


In [9]:
df['Speed_limit'].unique()

array([30, 40, 50, 20, 60, 70, 10, 15])

In [10]:
colsWithMissing = ['Carriageway_Hazards', '2nd_Road_Number', 'Road_Surface_Conditions']
rwIndxs = []

for i in colsWithMissing:
    rwIndxs = rwIndxs + data.loc[data[i] == -1].index.to_list()

data.drop(rwIndxs, inplace=True)
data.shape

(10679, 32)

In [11]:
#identify repeat vals/cols

def testIfIdentic(dat, col1, col2):
    flag = True
    similar = 0
    different = 0
    total = 0
    col1vals = []
    col2vals = []
    
    if(len(dat[col1].unique()) != len(dat[col2].unique())):
        vals = {0: False, 1: 0, 2: 0, 3: False}
        return vals
    
    for i in dat.index.tolist():
        x = dat.at[i, col1]
        y = dat.at[i, col2]
        
        if(x == -1 or y == -1):
            continue
        
        if(x in col1vals):
            indx = col1vals.index(x)
            if(col2vals[indx] != y):
                different += 1
                flag = False
            else:
                similar += 1
        else:
            col1vals.append(x)
            col2vals.append(y)
        total += 1
        
    vals = {0: flag, 1: similar, 2: different, 3: True}
    
    return vals



In [12]:
identicAttriDF = pd.DataFrame(columns=['col1', 'col2', 'isIdentic', 'similar', 'different'])

colNames = list(dict(data.dtypes).keys())
done = []
    
for a in colNames:
    for b in colNames:
        if (a == b or a in done):
            continue
        test = testIfIdentic(data, a, b)
        if (test[3] == False):
            continue
        vals = {'col1': a, 'col2': b, 'isIdentic': test[0], 'similar': test[1], 'different': test[2]}
        identicAttriDF = identicAttriDF.append(vals, ignore_index = True)
    done.append(a)
    
identicAttriDF.shape

(56, 5)

In [13]:
identicAttriDF.loc[identicAttriDF['isIdentic'] ==  True]

,col1,col2,isIdentic,similar,different
11,Local_Authority_(District),Local_Authority_(Highway),True,10676,0
15,Local_Authority_(Highway),Local_Authority_(District),True,10676,0


In [14]:
#drop police force atribut because all entries have same value
#drop Junction_control and 2nd_road_class beacuse they have over 20% missing entries

data.drop(columns=['Police_Force', 'Junction_Control', '2nd_Road_Class', 'Local_Authority_(District)'], inplace=True)

In [15]:
identicAttriDF.sort_values(by= ['similar'])

,col1,col2,isIdentic,similar,different
34,2nd_Road_Number,1st_Road_Number,False,817,9806
48,Road_Surface_Conditions,2nd_Road_Class,False,1432,6340
22,Speed_limit,2nd_Road_Class,False,1561,6211
27,Junction_Control,2nd_Road_Class,False,1562,6211
43,Light_Conditions,2nd_Road_Class,False,1569,6203
7,Number_of_Casualties,Day_of_Week,False,1624,9048
5,Number_of_Vehicles,Day_of_Week,False,1671,9001
28,Junction_Control,Light_Conditions,False,2835,4938
45,Weather_Conditions,Junction_Detail,False,3124,7546
23,Speed_limit,Light_Conditions,False,3188,7486


In [16]:
sim = testIfIdentic(df, 'Local_Authority_(District)', 'Local_Authority_(Highway)')
print(sim)

{0: True, 1: 145189, 2: 0, 3: True}


In [17]:
binary = {'Urban_or_Rural_Area'}
nomial = {'Did_Police_Officer_Attend_Scene_of_Accident', 'Local_Authority_(Highway)', '1st_Road_Class', 'Pedestrian_Crossing-Human_Control', 'Pedestrian_Crossing-Physical_Facilities', 'Junction_Detail', 'Road_Type'}
ordinal = {'Accident_Severity', 'Day_of_Week', 'Speed_limit'}
interval = {'DateVals', 'TimeVals', 'Number_of_Vehicles', 'Number_of_Casualties'}
ratio = {'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Latitude', 'Longitude'}
identifier = {'Accident_index'}

In [18]:
# clean data with outliers
data.shape

(10679, 28)

In [19]:
# finding outliers 
excludeCol = {'Accident_Index', 'Location_Easting_OSGR', 'Location_Northing_OSGR', 'Longitude', 'Latitude', 'Local_Authority_(Highway)', 'Local_Authority_(District)', 'LSOA_of_Accident_Location', '1st_Road_Number', '2nd_Road_Number'}
numericCol = {'Number_of_Casualties', 'Number_of_Vehicles', 'DateVals', 'TimeVals' }
categorCol = (set(data.columns.to_list()) - numericCol) - excludeCol

In [20]:
# outlier detectioction of numeric Cols: colName, std, mean, lower, upper, numOfOutliers
outNumData = pd.DataFrame(columns=['name', 'std', 'mean', 'lower', 'upper', 'numOfOutliers'])
outNumIndex = []

for i in numericCol:
    
    std = data[i].std()
    mean = data[i].mean()
    lower = mean - 3 * std
    upper = mean + 3 * std
    count = 0
    
    for row in data[i]:
        if ((row < lower) | (row > upper)):
            count += 1
            
    if (count == 0):
        continue
    
    vals = {'name':i, 'std':std, 'mean':mean, 'lower':lower, 'upper':upper, 'numOfOutliers':count}
    outNumData = outNumData.append(vals, ignore_index=True)
    
outNumData

,name,std,mean,lower,upper,numOfOutliers
0,Number_of_Casualties,0.563963,1.195898,-0.495990,2.887787,351
1,Number_of_Vehicles,0.616023,1.761026,-0.087044,3.609097,111


In [21]:
#finding Threshold for Categorical cols using numeric outlier results
numOfRows = data.shape[0]

thrshldCalc = [0, 0]
for i in outNumData['numOfOutliers']:
    thrshldCalc[0] += i
    thrshldCalc[1] += 1
    
thrshld = thrshldCalc[0] / thrshldCalc[1]

avgTPerc = thrshld / numOfRows
lwrTPerc = outNumData['numOfOutliers'].min() / numOfRows
print(numOfRows, avgTPerc, lwrTPerc)

10679 0.021631238880044947 0.010394231669631987


In [22]:
# outlier detectioction of categorical Cols: colName, value, rate
outCatData = pd.DataFrame(columns=['name', 'value', 'number', 'rate'])

percent = lwrTPerc

for i in categorCol:
    
    colValDict = dict(data[i].value_counts())
    
    for v in colValDict.keys():
        
        if(v == -1):
            continue
        
        
        if(colValDict[v] / numOfRows < percent):
            vals = {'name': i, 'value':v, 'number': colValDict[v], 'rate':(colValDict[v] / numOfRows) * 100}
            outCatData = outCatData.append(vals, ignore_index=True)

outCatData.sort_values(by=['name'], inplace=True)
outCatData

,name,value,number,rate
28,Accident_Severity,1,71,0.664856
29,Carriageway_Hazards,2,36,0.337110
33,Carriageway_Hazards,3,5,0.046821
32,Carriageway_Hazards,1,5,0.046821
31,Carriageway_Hazards,6,8,0.074913
30,Carriageway_Hazards,7,12,0.112370
10,Did_Police_Officer_Attend_Scene_of_Accident,3,63,0.589943
6,Junction_Detail,9,92,0.861504
7,Junction_Detail,5,71,0.664856
15,Light_Conditions,6,7,0.065549


In [23]:
# Oulier Categorical descriptive df: colName, numOfValues
outDescData = pd.DataFrame(columns=['name', 'flaggedVals', 'totalVals', 'flaggedPerc', 'dominant'])
percDict = dict.fromkeys(outCatData['name'].unique().tolist(), 0)

for i in outCatData['name'].unique().tolist():
    num = 0
    
    for p in outCatData.loc[outCatData['name'] == i]['rate']:
        percDict[i] += p
        num += 1
        
    g = anaDF1.loc[anaDF1['Name'] == i]['Unique']
    
    domi = False
    if(g.iloc[0] - num == 1):
        domi = True
    
    vals = {'name': i, 'flaggedVals': num, 'totalVals': g.iloc[0], 'flaggedPerc': percDict[i], 'dominant': domi}
    outDescData = outDescData.append(vals, ignore_index=True)

outDescData

,name,flaggedVals,totalVals,flaggedPerc,dominant
0,Accident_Severity,1,3,0.664856,False
1,Carriageway_Hazards,5,6,0.618035,True
2,Did_Police_Officer_Attend_Scene_of_Accident,1,3,0.589943,False
3,Junction_Detail,2,9,1.526360,False
4,Light_Conditions,3,5,0.543122,False
5,Pedestrian_Crossing-Human_Control,2,3,0.177919,True
6,Pedestrian_Crossing-Physical_Facilities,2,6,0.533758,False
7,Road_Surface_Conditions,3,5,0.861504,False
8,Road_Type,2,6,0.618035,False
9,Special_Conditions_at_Site,7,8,1.114337,True


In [24]:
unFlagCatCol = categorCol - set(outDescData['name'].to_list())
unFlagCatCol

{'1st_Road_Class', 'Day_of_Week', 'Urban_or_Rural_Area'}

In [25]:
#creating a dataframe with all outliers and removing them from the data dataframe

outRecords = pd.DataFrame(columns = data.columns.to_list())

datDict = dict(data.dtypes)

for key, val in datDict.items():
    if val == 'int64' or val == 'float64':
        outRecords[key] = pd.to_numeric(outRecords[key])
        
#removing numeric outliers
for i in outNumData['name'].unique():
    upper = outNumData.loc[outNumData['name'] == i]['upper'].tolist()[0]
    lower = outNumData.loc[outNumData['name'] == i]['lower'].tolist()[0]
    outs = data.loc[(data[i] > upper) | (data[i] < lower)]
    data.drop(index = data.index[(data[i] > upper) | (data[i] < lower)].tolist(), inplace = True)
    outRecords = pd.concat([outRecords, outs])
    
    
print(data.shape)
print(outRecords.shape)
        

(10235, 28)
(444, 28)


In [26]:
#removing categorical outliers
for i in outCatData['name'].unique():
    vals = outCatData.loc[outCatData['name'] == i]['value'].tolist()
    for v in vals:
        outs = data.loc[data[i] == v]
        data.drop(index = data.index[data[i] == v].tolist(), inplace = True)
        outRecords = pd.concat([outRecords, outs])
        
print(data.shape)
print(outRecords.shape)

(9374, 28)
(1305, 28)


In [27]:
anaDF2 = analyzeDF(data)
anaDF2

(28, 8)


,Name,Type,Empty,Full,Ratio,Min,Max,Unique
0,Accident_Index,object,0,9374,0.0,Nan,Nan,9374
1,Location_Easting_OSGR,int64,0,9374,0.0,510800,535190,1815
2,Location_Northing_OSGR,int64,0,9374,0.0,175590,193940,1261
3,Longitude,float64,0,9374,0.0,-0.401384,-0.049164,6445
4,Latitude,float64,0,9374,0.0,51.46535,51.632397,6289
5,Accident_Severity,int64,0,9374,0.0,2,3,2
6,Number_of_Vehicles,int64,0,9374,0.0,1,3,3
7,Number_of_Casualties,int64,0,9374,0.0,1,2,2
8,Day_of_Week,int64,0,9374,0.0,1,7,7
9,Local_Authority_(Highway),object,0,9374,0.0,Nan,Nan,3


In [28]:
#droping all columns that only have 1 unique value
#the value in all the columsn was 0 signifying none, view document for extra info
print(data['Carriageway_Hazards'].unique())
print(data['Special_Conditions_at_Site'].unique())
print(data['Pedestrian_Crossing-Human_Control'].unique())

categorCol.remove('Carriageway_Hazards')
categorCol.remove('Pedestrian_Crossing-Human_Control')
categorCol.remove('Special_Conditions_at_Site')

[0]
[0]
[0]


In [29]:
data.drop(columns=['Carriageway_Hazards', 'Special_Conditions_at_Site', 'Pedestrian_Crossing-Human_Control'], inplace=True)
print(data.shape)

(9374, 25)


In [30]:
#exporting outlier free and clean data

data.to_csv(path_or_buf='/Users/othmanetazi/Desktop/DMA_ousework/processedData.csv', index = False)

FileNotFoundError: [Errno 2] No such file or directory: '/Users/othmanetazi/Desktop/DMA_ousework/processedData.csv'

In [ ]:
list(dict(data.dtypes).keys())

In [ ]:
def bucketNumeric(datf, cols, bucket):
    outDF = pd.DataFrame()
    #refDF = pd.DataFrame()
    for col in datf.columns.to_list():
        if (col in cols):  
            spread = datf[col].max() - datf[col].min()
            mini = datf[col].min()
            vals = []
            for i in datf[col]:
                for x in range(bucket):
                    if( i - mini > (spread / bucket) * (x + 1)):
                        continue
                    else:
                        vals.append(x)
                        break
            outDF[col] = vals
        else:
            outDF[col] = datf[col]
            
    return outDF

In [ ]:
len(data['DateVals'].unique())

In [ ]:
def oneHotDF(datf, cols):
    outDF = pd.DataFrame()
    #refDF = pd.DataFrame()
    for col in datf.columns.to_list():
        if (col in cols):  
            colVals = list(datf[col].value_counts().keys())
            valList = {}
            for val in colVals:
                name = col + '_' + str(val)
                valList[name] = []
            for row in datf[col]:
                for i in colVals:
                    name = col + '_' + str(i)
                    if(i == row):
                        valList[name].append(1)
                    else:
                        valList[name].append(0)
            for k, v in valList.items():
                outDF[k] = v
        else:
            outDF[col] = datf[col]
            
    return outDF
        


In [ ]:
list(data['Accident_Severity'].value_counts().keys())

In [ ]:
impoCat = ['Accident_Severity', 'Day_of_Week', '1st_Road_Class', 'Junction_Detail', 'Speed_limit', 'Road_Type', 'Pedestrian_Crossing-Physical_Facilities', 'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Urban_or_Rural_Area', 'Did_Police_Officer_Attend_Scene_of_Accident']
impoNum = ['Number_of_Casulaties', 'Number_of_Vehicles', 'DateVals', 'TimeVals']

In [ ]:
oheDF = oneHotDF(data, impoCat)
oheDF.to_csv(path_or_buf='/Users/othmanetazi/Desktop/DMA_ousework/processedOneHotData.csv', index = False)

In [ ]:
oheDF.dtypes